# **Agents for Real — Plan. Act. Deliver.**

**LangChain · LangFlow · MCP (safe)**

> 25‑minute live tour + code

### Agenda
- Why agents (vs. prompts)
- Patterns: ReAct, Planner/Executor, Graph
- LangChain vs LangFlow: code vs visual
- LangGraph (stateful graphs)
- MCP: safe tools
- Live demo + micro‑eval
- Risks & guardrails

## Big idea
**Tools > Prompts.** Agents plan, call tools, and verify.

**Production focus:** versioned graphs, safe tools, evals, observability.

## Agentic patterns
- **ReAct** — think/act/observe
- **Planner/Executor** — global plan across steps
- **Graph / Multi‑agent** — parallel skills, routing, shared state

## LangChain vs LangFlow (practical)
- **LangChain** (code‑first): LCEL, tests, CI/CD
- **LangFlow** (visual): DAGs, tweak params, export JSON, REST/MCP endpoints
- **Workflow**: ideate in LangFlow → codify in LangChain

## Live demo — what you'll run
1) Build a tiny **ReAct** agent (retriever + calculator)
2) Call a **LangFlow** flow via REST
3) Attach **MCP** tools to the same agent
4) **LangGraph** mini‑pipeline (retrieve → answer + optional router)

### Setup (one time)
```bash
pip install -U langchain langchain-openai langchain-community langchain-ollama pandas requests matplotlib
pip install -U langflow "mcp[cli]" langchain-mcp-adapters langgraph
# (Optional) Ollama: install and `ollama pull llama3.1:8b`
```
In the next cell, set `BACKEND = "OPENAI"` or `"OLLAMA"`.

In [ ]:
# Backend config — set ONE line
BACKEND = "OLLAMA"  # or "OPENAI"
MODEL_OPENAI = "gpt-4o-mini"
MODEL_OLLAMA = "llama3.1:8b"  # ensure it's pulled if using Ollama

import os, json, re, math, requests, ast, operator
from typing import List, Dict, Any

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

try:
    if BACKEND == "OPENAI":
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=MODEL_OPENAI, temperature=0.2)
    else:
        try:
            from langchain_community.chat_models import ChatOllama
        except Exception:
            from langchain_community.llms import Ollama as ChatOllama
        llm = ChatOllama(model=MODEL_OLLAMA)
except Exception as e:
    raise RuntimeError(f"Could not initialize LLM for BACKEND={BACKEND}: {e}")


In [ ]:
# Tiny policy corpus + tools
POLICIES = [
    {"id": "TravelPlus-2024", "text": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."},
    {"id": "DeviceCare-Pro", "text": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."},
    {"id": "FleetAssist", "text": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."},
]

def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

def search_policies(query: str) -> str:
    hits = simple_search(query, top_k=3)
    return json.dumps({"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]})

OPS = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv,
       ast.Pow: operator.pow, ast.USub: operator.neg, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv}
def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.BinOp): return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp): return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")
def calculator(expression: str) -> str:
    try:
        node = ast.parse(expression, mode='eval').body
        return str(_eval(node))
    except Exception as e:
        return f"Error: {e}"

tools = [
    Tool(name="search_policies", func=search_policies, description="Search a tiny policy corpus. Returns JSON."),
    Tool(name="calculator", func=calculator, description="Evaluate arithmetic like '2*(1500-200)'. Returns a number as text.")
]


In [ ]:
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True, handle_parsing_errors=True)

demo_questions = [
    "My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?",
    "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
    "Does FleetAssist cover personal belongings inside a rental car?"
]
print("Demo questions:", *demo_questions, sep="\n- ")


In [ ]:
agent.invoke(demo_questions[0])

In [ ]:
agent.invoke(demo_questions[1])

In [ ]:
agent.invoke(demo_questions[2])

In [ ]:
import pandas as pd
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    def display_dataframe_to_user(name, df):
        print(f"\n[name={name}]\n{df}")

def contains_all(text: str, phrases: list[str]) -> bool:
    low = text.lower()
    return all(p.lower() in low for p in phrases)

tests = [
    {"q": demo_questions[0], "must": ["covered", "theft", "police", "48"]},
    {"q": demo_questions[2], "must": ["not", "unless", "endorsed"]},
]
rows = []
for t in tests:
    out = agent.invoke(t["q"])
    ans = out["output"] if isinstance(out, dict) and "output" in out else str(out)
    rows.append({"question": t["q"], "passed": contains_all(ans, t["must"]), "answer": ans})

df = pd.DataFrame(rows)
display_dataframe_to_user("LangChain agent micro‑eval", df)
df


## LangFlow — visual builder
- Start: `langflow run --host 127.0.0.1 --port 7860`
- Build: Chat Model → Prompt → (optional) Tool → Agent → **Play**
- Call from Python via REST (next cell)

In [ ]:
import requests, json

def run_langflow(flow_id: str, user_input: str, base_url: str = "http://127.0.0.1:7860/api/v1"):
    url = f"{base_url}/run/{flow_id}"
    payload = {"input_value": user_input, "output_type": "chat", "input_type": "chat"}
    r = requests.post(url, json=payload, timeout=60)
    r.raise_for_status()
    data = r.json()
    try:
        return data["outputs"][0]["outputs"][0]["results"]["message"]["text"]
    except Exception:
        return json.dumps(data, indent=2)

# print(run_langflow("YOUR_FLOW_ID", "Hello from the notebook!"))


## MCP — safe tool server
Standardize tools; safer by design (allow‑lists, validation). We'll write a small server and attach it.

In [ ]:
mcp_server_code = r'''
from __future__ import annotations
from mcp.server.fastmcp import FastMCP
import re, ast, operator, json

OPS = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv,
       ast.Pow: operator.pow, ast.USub: operator.neg, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv}
def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.BinOp): return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp): return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")

def safe_calculator(expression: str) -> str:
    node = ast.parse(expression, mode='eval').body
    return str(_eval(node))

POLICIES = [
    {"id": "TravelPlus-2024", "text": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."},
    {"id": "DeviceCare-Pro", "text": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."},
    {"id": "FleetAssist", "text": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."},
]
def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

mcp = FastMCP("SR-Policies", stateless_http=True)
MAX_Q = 200
BANNED = re.compile(r"(?i)(rm\s|-rf|\bimport\b|__|eval\(|exec\()")

@mcp.tool()
def secure_search_policies(query: str) -> dict:
    if not query or len(query) > MAX_Q or BANNED.search(query or ""):
        return {"error": "query_rejected"}
    hits = simple_search(query, top_k=3)
    return {"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]}

@mcp.tool()
def safe_calc(expression: str) -> str:
    if len(expression or "") > 100 or BANNED.search(expression or ""):
        return "error: expression_rejected"
    try:
        return safe_calculator(expression)
    except Exception as e:
        return f"error: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http", host="127.0.0.1", port=3001)
'''
server_path = "/mnt/data/mcp_safe_server.py"
with open(server_path, "w", encoding="utf-8") as f:
    f.write(mcp_server_code)
print("Wrote MCP server to:", server_path)


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import initialize_agent, AgentType
import asyncio

async def load_mcp_toolset():
    client = MultiServerMCPClient({
        "sr_policies": {"transport": "streamable_http", "url": "http://127.0.0.1:3001/mcp/"}
    })
    return await client.get_tools()

try:
    mcp_tools = asyncio.run(load_mcp_toolset())
    print("MCP tools:", [t.name for t in mcp_tools])
except Exception as e:
    print("Start the MCP server first. Error:", e)


## LangGraph — stateful agent graphs
- **State schema** → **nodes** → **edges**
- Optional **checkpointer** (threaded memory)
- Below: a minimal retrieve→answer pipeline using our existing `llm` + retriever.


### Install (if needed)
```bash
pip install -U langgraph
```


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
try:
    from langgraph.checkpoint import MemorySaver
    checkpointer = MemorySaver()
except Exception:
    checkpointer = None

class AgentState(TypedDict, total=False):
    question: str
    context: str
    answer: str

def retrieve_node(state: AgentState) -> AgentState:
    import json as _json
    raw = search_policies(state.get("question", ""))
    data = _json.loads(raw)
    snippets = [r.get("snippet","") for r in data.get("results", [])]
    return {"context": "\n\n".join(snippets) if snippets else "No policy hits."}

def answer_node(state: AgentState) -> AgentState:
    q = state.get("question",""); cx = state.get("context","")
    prompt = ("Use ONLY the policy snippets to answer. If insufficient, say so.\n\n"
              f"Policy snippets:\n{cx}\n\nQuestion: {q}\n\nAnswer:")
    out = llm.invoke(prompt)
    text = getattr(out, "content", str(out))
    return {"answer": text}

graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve_node)
graph.add_node("answer", answer_node)
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "answer")
graph.add_edge("answer", END)
app = graph.compile(checkpointer=checkpointer) if checkpointer else graph.compile()

print(app.invoke({"question": "Does FleetAssist cover belongings in a rental car?"}).get("answer"))


In [ ]:
# Optional: add a simple router for calc-like questions
import re

def route(state: AgentState) -> str:
    q = state.get("question","")
    return "calc" if re.search(r"\bCHF\b|\d+\s*[+\-*/]", q) else "retrieve"

def calc_node(state: AgentState) -> AgentState:
    q = state.get("question","")
    m = re.search(r"(\d+[\d\s+\-*/().]*)", q)
    expr = m.group(1) if m else None
    out = calculator(expr) if expr else "Error: no expression found"
    return {"context": f"Calculated: {expr} = {out}", "answer": str(out)}

g2 = StateGraph(AgentState)
g2.add_node("retrieve", retrieve_node)
g2.add_node("calc", calc_node)
g2.add_node("answer", answer_node)
g2.add_conditional_edges(START, route, {"retrieve": "retrieve", "calc": "calc"})
g2.add_edge("retrieve", "answer")
g2.add_edge("calc", "answer")
g2.add_edge("answer", END)
app_routed = g2.compile()

print(app_routed.invoke({"question": "If a laptop worth CHF 1,800 is damaged twice, what's the total?"}).get("answer"))


## Closing
Agents are **tools-first** systems. Ship‑ready means:
- Versioned graphs
- Safe MCP/allow‑listed tools
- Eval + observability

**Thanks!**